In [1]:
import matplotlib
import pandas as pd
import numpy as np
import time
from datetime import datetime
import matplotlib.pyplot as plt
from sklearn.naive_bayes import *
from sklearn.linear_model import LinearRegression
from re import sub
def mdy_to_ymd(d):
    return time.mktime(datetime.strptime(d, '%d-%b-%y').timetuple())

class Beautify:

    @staticmethod
    def beautify(path):
        dataset = pd.read_csv(path)
        dataset.columns = ["date", "price", "open", "high", "low", "vol", "change"]
        dataset["date"] = dataset.apply(lambda row: mdy_to_ymd(row['date']), axis=1)
        for col in ["price", "open", "high", "low", "vol", "change"]:
            dataset[col] = dataset.apply(
                lambda row: (lambda x: 0 if x == '-' else float(x))(sub(r'[^\d.-]', '', row[col])),
                axis=1)
        dataset = dataset.sort_values(["date"])
        return dataset


pd.set_option('display.float_format', lambda x: '%.0f' % x)

In [2]:
dataset = Beautify.beautify("dataset.csv")

dataset = dataset[["date","price","change"]]
dataset["change"] = dataset.apply(lambda x: -1 if x["change"] <= 0 else 1,axis=1)

In [3]:
for i in [1,2,3,5,7,10,12,15,20,30,45]:
    dataset[f"s_{i}"] = dataset["price"].shift(i,fill_value=0)

In [4]:
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
model =  Lasso()

In [5]:
TRAIN = time.mktime(datetime.strptime("20-01-02", '%y-%m-%d').timetuple())
train = dataset[dataset.date < TRAIN]
test = dataset[dataset.date >= TRAIN]

In [6]:
train.drop("date",axis=1,inplace=True)
test.drop("date",axis=1,inplace=True)

E:\Anaconda2021\lib\site-packages\pandas\core\frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [7]:
pred = model.fit(train.drop("price",axis=1),train.price).predict(test.drop("price",axis=1))

E:\Anaconda2021\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 61794532.24392402, tolerance: 4106899.6988909775
  model = cd_fast.enet_coordinate_descent(


In [8]:
res = pd.DataFrame({"index": np.arange(len(pred)), "real": test.price, "pred": pred})

In [9]:
res[(res.pred * 0.95 <= res.real) & (res.pred * 1.05 >= res.real)].shape[0] / res.shape[0] * 100

86.83127572016461

In [10]:
from sklearn.metrics import mean_squared_error
mean_squared_error(res.pred,res.real)

1101097.1524989677